In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
import pandas as pd

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths for saving precomputed features or loading cached features
FEATURE_DIR = "feat_embedding_1_10"
os.makedirs(FEATURE_DIR, exist_ok=True)

# Transformation pipeline for EfficientNet-B3
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# EfficientNet-B3 model without the final layer for feature extraction
class EfficientNetB3WithoutFC(nn.Module):
    def __init__(self):
        super(EfficientNetB3WithoutFC, self).__init__()
        original_model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.DEFAULT)
        self.features = nn.Sequential(*list(original_model.children())[:-1])  # Remove FC layer
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Add adaptive pooling for feature extraction

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        return x.view(x.size(0), -1)  # Flatten the output

feature_extractor = EfficientNetB3WithoutFC().to(device)
feature_extractor.eval()

EfficientNetB3WithoutFC(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
              (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (s

In [17]:
# Function to extract and cache features in batches

def load_or_compute_features(images, dataset_name, transform, model, batch_size=64):
    cache_path = os.path.join(FEATURE_DIR, f"{dataset_name}_features.npy")
    if os.path.exists(cache_path):
        print(f"Loading cached features for {dataset_name}...")
        return np.load(cache_path)

    # Create DataLoader for batch processing
    dataset = torch.utils.data.TensorDataset(torch.stack([transform(img) for img in images]))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    features = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Extracting features for {dataset_name}"):
            batch = batch[0].to(device)  # Access data and move to device
            feature_batch = model(batch).cpu().numpy()
            features.append(feature_batch)

    features = np.vstack(features)
    np.save(cache_path, features)  # Cache features for future reuse
    return features



# Define LwP Classifier using Mahalanobis distance
class LwPClassifierMahalanobis:
    def __init__(self):
        self.prototypes = None
        self.covariances = None
        self.classes = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.prototypes = {}
        self.covariances = {}

        for label in self.classes:
            class_features = X[y == label]
            self.prototypes[label] = np.mean(class_features, axis=0)

            # Compute covariance matrix with shrinkage to handle singularity
            cov_matrix = np.cov(class_features, rowvar=False)
            cov_matrix += np.eye(cov_matrix.shape[0]) * 1e-6  # Regularization
            self.covariances[label] = np.linalg.inv(cov_matrix)  # Inverse covariance matrix

    def mahalanobis_distance(self, x, prototype, cov_inv):
        delta = x - prototype
        return np.sqrt(delta.T @ cov_inv @ delta)

    def predict(self, X):
        predictions = []
        for x in X:
            # Calculate Mahalanobis distance to each prototype and choose the nearest
            distances = {
                label: self.mahalanobis_distance(x, self.prototypes[label], self.covariances[label])
                for label in self.classes
            }
            predictions.append(min(distances, key=distances.get))
        return np.array(predictions)

    def update(self, X, y):
        # Update prototypes and covariance matrices with new data
        for label in self.classes:
            class_features = X[y == label]
            if label in self.prototypes:
                # Combine old and new prototypes and covariances
                old_mean = self.prototypes[label]
                old_cov_inv = self.covariances[label]
                new_mean = np.mean(class_features, axis=0)

                # Update mean using weighted average
                self.prototypes[label] = 0.85 * old_mean + 0.15 * new_mean

                # Update covariance using weighted average
                new_cov_matrix = np.cov(class_features, rowvar=False) + np.eye(class_features.shape[1]) * 1e-6
                self.covariances[label] = 0.85 * old_cov_inv + 0.15 * np.linalg.inv(new_cov_matrix)


In [18]:
# Update process_datasets_sequentially to consistently use caching

def process_datasets_sequentially(train_paths, eval_paths, lwp):

    for i, (train_path, eval_path) in enumerate(zip(train_paths[1:], eval_paths[1:]), start=2):
        print(f"\nProcessing training dataset {i}...")
        train_data = torch.load(train_path)
        train_images = train_data['data']

        # Load or compute training features
        train_features = load_or_compute_features(train_images, f"train_{i}", transform, feature_extractor)

        # Predict pseudo-labels for the training dataset
        pseudo_labels = lwp.predict(train_features)

        # Update the LwP Classifier
        lwp.update(train_features, pseudo_labels)


In [19]:
#Paths for train and eval datasets
train_paths = [f"dataset/part_one_dataset/train_data/{i}_train_data.tar.pth" for i in range(1, 11)]
eval_paths = [f"dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]

# Load the first training dataset
train_data_1 = torch.load(train_paths[0])
D1_images = train_data_1['data']
D1_labels = train_data_1['targets']

# Extract features for the first dataset
D1_features = load_or_compute_features(D1_images, "train_1", transform, feature_extractor)

# Initialize the LwP classifier with Euclidean distance
lwp = LwPClassifierMahalanobis()
lwp.fit(D1_features, np.array(D1_labels))

# Process datasets sequentially and get the detailed summary
detailed_summary_df = process_datasets_sequentially(train_paths, eval_paths, lwp)

Loading cached features for train_1...

Processing training dataset 2...
Loading cached features for train_2...

Processing training dataset 3...
Loading cached features for train_3...

Processing training dataset 4...
Loading cached features for train_4...

Processing training dataset 5...
Loading cached features for train_5...

Processing training dataset 6...
Loading cached features for train_6...

Processing training dataset 7...
Loading cached features for train_7...

Processing training dataset 8...
Loading cached features for train_8...

Processing training dataset 9...
Loading cached features for train_9...

Processing training dataset 10...
Loading cached features for train_10...


In [20]:
print(lwp.prototypes)

{np.int64(0): array([ 0.05449521,  0.00056987,  0.07043332, ..., -0.11346348,
        0.31664404, -0.10727898], dtype=float32), np.int64(1): array([ 0.08313376,  0.10209099,  0.07645009, ..., -0.01488356,
        0.0746142 ,  0.02628328], dtype=float32), np.int64(2): array([ 0.03387743, -0.01180812,  0.0579953 , ..., -0.03884554,
        0.14813577,  0.1531308 ], dtype=float32), np.int64(3): array([0.11995536, 0.04611782, 0.04923675, ..., 0.01600114, 0.1440477 ,
       0.01228173], dtype=float32), np.int64(4): array([ 0.09725423, -0.03132486,  0.02224294, ...,  0.00401522,
        0.1532101 , -0.02752708], dtype=float32), np.int64(5): array([ 0.14929634,  0.06586424,  0.000548  , ..., -0.04714416,
        0.02317119,  0.09694136], dtype=float32), np.int64(6): array([-0.0418111 , -0.04189126,  0.08957747, ..., -0.07926816,
        0.06450835,  0.10742693], dtype=float32), np.int64(7): array([ 0.20204225,  0.0287745 ,  0.18875253, ..., -0.09134164,
        0.10140918, -0.06515303], dtype

In [21]:
class LwPClassifierEuclidean:
    def __init__(self, prototypes):
        self.prototypes = prototypes  # Initialize with prototypes from "mean.pkl"
        self.classes = list(prototypes.keys()) if prototypes else []

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.prototypes = {}

        for label in self.classes:
            class_features = X[y == label]
            self.prototypes[label] = np.mean(class_features, axis=0)

    def euclidean_distance(self, x, prototype):
        return np.linalg.norm(x - prototype)

    def predict(self, X):
        predictions = []
        for x in X:
            # Calculate Euclidean distance to each prototype and choose the nearest
            distances = {label: self.euclidean_distance(x, self.prototypes[label]) for label in self.classes}
            predictions.append(min(distances, key=distances.get))
        return np.array(predictions)

    def update(self, X, y):
        for label in np.unique(y):
            class_features = X[y == label]
            if label in self.prototypes:
                # Combine old and new prototypes
                old_mean = self.prototypes[label]
                new_mean = np.mean(class_features, axis=0)

                # Weighted update for prototype
                self.prototypes[label] = 0.85 * old_mean + 0.15 * new_mean
            else:
                # Add a new class prototype if not previously seen
                self.prototypes[label] = np.mean(class_features, axis=0)

In [22]:
# Updated function to handle separate feature directory for Task 2
def load_or_compute_features_task2(images, dataset_name, transform, model, feature_dir, batch_size=32):
    cache_path = os.path.join(feature_dir, f"{dataset_name}_features.npy")
    if os.path.exists(cache_path):
        print(f"Loading cached features for {dataset_name} from Task 2 directory...")
        return np.load(cache_path)

    # Create DataLoader for batch processing
    dataset = torch.utils.data.TensorDataset(torch.stack([transform(img) for img in images]))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    features = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Extracting features for {dataset_name}"):
            batch = batch[0].to(device)  # Access data and move to device
            feature_batch = model(batch).cpu().numpy()
            features.append(feature_batch)

    features = np.vstack(features)
    np.save(cache_path, features)  # Cache features for future reuse
    return features

In [23]:
# Function to process datasets for Task 2 with correct paths and adjusted dataset naming
def process_task2_with_comprehensive_evaluation(train_paths_task2, eval_paths_task1, eval_paths_task2, lwp, feature_dir_task2):
    summary = []  # Store evaluation accuracies for all datasets from 1 to i

    # Combine Task 1 and Task 2 evaluation paths
    eval_paths = eval_paths_task1 + eval_paths_task2

    for i, train_path in enumerate(train_paths_task2, start=1):  # Start from 11th dataset for part_two_dataset
        print(f"\nProcessing training dataset D{i+10}...")
        train_data = torch.load(train_path)
        train_images = train_data['data']

        # Load or compute features for the training dataset using Task 2 directory
        train_features = load_or_compute_features_task2(train_images, f"train_{i}", transform, feature_extractor, feature_dir_task2)

        # Predict pseudo-labels using the LwP classifier
        pseudo_labels = lwp.predict(train_features)

        # Update the LwP classifier with the pseudo-labeled data
        lwp.update(train_features, pseudo_labels)

        # Evaluate on all evaluation datasets from 1 to i
        accuracies_for_current_training = []
        print(f"\nEvaluating updated classifier on datasets 1 to {i+10}...")
        for j in range(1, i + 11):  # Loop through all previous evaluation datasets (1 to i)
            eval_data_j = torch.load(eval_paths[j - 1])
            eval_images_j = eval_data_j['data']
            eval_labels_j = eval_data_j['targets']

            if j <= 10:
              eval_features_j = load_or_compute_features_task2(eval_images_j, f"eval_{j}", transform, feature_extractor, FEATURE_DIR)
            else:
              eval_features_j = load_or_compute_features_task2(eval_images_j, f"eval_{j-10}", transform, feature_extractor, feature_dir_task2)
            # Predict labels and calculate accuracy
            eval_predictions_j = lwp.predict(eval_features_j)
            eval_accuracy_j = accuracy_score(eval_labels_j, eval_predictions_j)
            print(f"Accuracy on evaluation dataset D̂{j}: {eval_accuracy_j * 100:.2f}%")
            accuracies_for_current_training.append(eval_accuracy_j * 100)

        # Append accuracies for this training step to the summary
        while len(summary) < (i+10) - 10:  # Ensure summary is aligned for Task 2 (starting at D11)
            summary.append([])
        summary[(i+10) - 11] = accuracies_for_current_training

    # Print final detailed summary for Task 2
    print("\nDetailed Summary of Task 2 Evaluation Accuracies:")
    df_summary = pd.DataFrame(summary)
    df_summary.index = [f"After Training {i}" for i in range(11, 11 + len(summary))]
    df_summary.columns = [f"Eval Dataset {j}" for j in range(1, df_summary.shape[1] + 1)]
    print(df_summary)
    return df_summary


# Paths for Task 2 datasets
FEATURE_DIR_TASK2 = "feat_embedding_11_20"  # Separate directory for Task 2 features
os.makedirs(FEATURE_DIR_TASK2, exist_ok=True)

# Paths for training datasets (Task 2 starts with 11th dataset as 1 in part_two_dataset)
train_paths_task2 = [f"dataset/part_two_dataset/train_data/{i}_train_data.tar.pth" for i in range(1, 11)]

# Paths for evaluation datasets
eval_paths_task1 = [f"dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]
eval_paths_task2 = [f"dataset/part_two_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]

# Initialising an lwp euclidean model with the previous prototypes
lwp_euclidean = LwPClassifierEuclidean(prototypes=lwp.prototypes)

# Process Task 2 datasets with comprehensive evaluation
task2_comprehensive_summary_df = process_task2_with_comprehensive_evaluation(train_paths_task2, eval_paths_task1, eval_paths_task2, lwp_euclidean, FEATURE_DIR_TASK2)


Processing training dataset D11...
Loading cached features for train_1 from Task 2 directory...

Evaluating updated classifier on datasets 1 to 11...
Loading cached features for eval_1 from Task 2 directory...
Accuracy on evaluation dataset D̂1: 86.80%
Loading cached features for eval_2 from Task 2 directory...
Accuracy on evaluation dataset D̂2: 88.56%
Loading cached features for eval_3 from Task 2 directory...
Accuracy on evaluation dataset D̂3: 88.88%
Loading cached features for eval_4 from Task 2 directory...
Accuracy on evaluation dataset D̂4: 88.52%
Loading cached features for eval_5 from Task 2 directory...
Accuracy on evaluation dataset D̂5: 88.08%
Loading cached features for eval_6 from Task 2 directory...
Accuracy on evaluation dataset D̂6: 88.24%
Loading cached features for eval_7 from Task 2 directory...
Accuracy on evaluation dataset D̂7: 87.84%
Loading cached features for eval_8 from Task 2 directory...
Accuracy on evaluation dataset D̂8: 88.12%
Loading cached features f